In [1]:
from datasets import load_dataset
ds = load_dataset('eth-dl-rewards/pref-data-math', split='eval')

In [ ]:
ds.num_rows

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

MODEL = 'eth-dl-rewards/internlm2-7b-reward-math-30k'
model = AutoModel.from_pretrained(MODEL, trust_remote_code=True, torch_dtype=torch.bfloat16, device_map='cuda')

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

In [4]:
def get_reward(prompt, solution):
  messages = [
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": solution}
  ]
  encoded = tokenizer.apply_chat_template(messages, tokenize=False)
  with torch.no_grad():
    inputs = tokenizer(encoded, return_tensors='pt', truncation=True, max_length=3500).to('cuda')
    print(inputs['input_ids'].shape)
    outputs = model(**inputs)
    reward = outputs.logits[0][0].item()
    del inputs
    del outputs
    gc.collect()
    torch.cuda.empty_cache()

    return reward


In [ ]:
from tqdm import tqdm
import gc
correct = 0
total = 0
gaps = []
model.eval()
for problem, accepted, rejected in tqdm(zip(ds['problem'], ds['accepted'], ds['rejected']), total=len(ds), desc='Evaluating'):
  reward_accepted = get_reward(problem, accepted)
  reward_rejected = get_reward(problem, rejected)
  gaps.append(reward_accepted - reward_rejected)
  if reward_accepted > reward_rejected:
    correct += 1
  total += 1

  if total % 100 == 0:
    print(f"Correct: {correct}/{total} ({correct/total*100:.2f}%)")
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
print(f"Correct: {correct}/{total} ({correct/total*100:.2f}%)")

In [ ]:
from tqdm import tqdm
import numpy as np
import gc
import torch

def get_rewards_batch(batch):
    prompts, solutions = zip(*batch)
    messages = [
        [{"role": "user", "content": prompt}, {"role": "assistant", "content": solution}]
        for prompt, solution in zip(prompts, solutions)
    ]
    encoded = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(encoded, return_tensors='pt', truncation=True, max_length=4096, padding=True).to('cuda')
    with torch.no_grad():
        outputs = model(**inputs)
        rewards = outputs.logits[:, 0].tolist()
        del inputs, outputs
        return rewards

def precompute_rewards(ds, batch_size=10):
    rewards = []
    for i in tqdm(range(0, len(ds), batch_size), desc='Precomputing Rewards'):
        batch = list(zip(ds['problem'][i:i + batch_size], ds['accepted'][i:i + batch_size], ds['rejected'][i:i + batch_size]))
        accepted_batch = [(problem, accepted) for problem, accepted, _ in batch]
        rejected_batch = [(problem, rejected) for problem, _, rejected in batch]

        reward_accepted = get_rewards_batch(accepted_batch)
        reward_rejected = get_rewards_batch(rejected_batch)

        batch_rewards = list(zip(reward_accepted, reward_rejected))
        rewards.extend(batch_rewards)

        gc.collect()
        torch.cuda.empty_cache()

    return rewards

def evaluate_from_precomputed(rewards):
    correct = 0
    total = 0
    gaps = []

    for reward_accepted, reward_rejected in rewards:
        gap = reward_accepted - reward_rejected
        gaps.append(gap)
        if reward_accepted > reward_rejected:
            correct += 1
        total += 1

    return correct, total, gaps

# Example usage:
# ds should be a list of tuples in the form [(problem, accepted, rejected), ...]
batch_size = 8
precomputed_rewards = precompute_rewards(ds, batch_size=batch_size)
correct, total, gaps = evaluate_from_precomputed(precomputed_rewards)
print(f"Final Results: Correct: {correct}/{total} ({correct/total*100:.2f}%)")


In [ ]:
from transformers import AutoConfig, AutoModel
config = AutoConfig.from_pretrained("internlm/internlm2-7b")

In [ ]:
config.auto_map['AutoModel'] = 'internlm/internlm2-7b--modeling_internlm2.InternLM2ForSequenceClassification'

In [3]:
from datasets import load_dataset

In [4]:
ds = load_dataset('eth-dl-rewards/pref-data-code')

README.md:   0%|          | 0.00/357 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/39.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/69363 [00:00<?, ? examples/s]

In [7]:
# we take first 60k examples for train and rest for eval
train_ds = ds['train'].select(range(60000))
eval_ds = ds['train'].select(range(60000, ds['train'].num_rows))

In [11]:
new_ds = {
    'train': train_ds,
    'eval': eval_ds
}
from datasets import DatasetDict
new_ds = DatasetDict(new_ds)
new_ds

DatasetDict({
    train: Dataset({
        features: ['problem', 'accepted', 'rejected'],
        num_rows: 60000
    })
    eval: Dataset({
        features: ['problem', 'accepted', 'rejected'],
        num_rows: 9363
    })
})

In [12]:
new_ds.push_to_hub('eth-dl-rewards/pref-data-code')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/60 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/eth-dl-rewards/pref-data-code/commit/9619a3edd91cea6b93e96024c3da75ffb374111c', commit_message='Upload dataset', commit_description='', oid='9619a3edd91cea6b93e96024c3da75ffb374111c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/eth-dl-rewards/pref-data-code', endpoint='https://huggingface.co', repo_type='dataset', repo_id='eth-dl-rewards/pref-data-code'), pr_revision=None, pr_num=None)